In [1]:
#Step 1: Install Required Libraries

!pip install opencv-python-headless



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#Import Libraries

import cv2
import numpy as np
import requests


In [5]:
# Define URLs for the pre-trained models

age_model = "https://www.dropbox.com/scl/fi/eva2roklpqn0lh52mk5j2/Copy-age_net.caffemodel?rlkey=nt9d615zw0tupmqsrgit635y5&st=cdrdqk63&dl=0"
age_proto = "age_deploy.prototxt"
gender_model = "https://www.dropbox.com/scl/fi/ynm8706p7mf1kvyj3ppf3/gender_net.caffemodel?rlkey=c4mjwtr4iuzpstwgeowr7tuk7&st=w4xpdcmf&dl=0"
gender_proto = "gender_deploy.prototxt"

# Path to OpenCV pre-trained face detector
face_model = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


In [7]:
#Download the Models

!wget $age_model -O age.caffemodel
!wget $age_proto -O age_deploy.prototxt
!wget $gender_model -O gender.caffemodel
!wget $gender_proto -O gender_deploy.prototxt


'wget' is not recognized as an internal or external command,
operable program or batch file.
'st' is not recognized as an internal or external command,
operable program or batch file.
'dl' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'st' is not recognized as an internal or external command,
operable program or batch file.
'dl' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
# Load age and gender prediction models

age_net = cv2.dnn.readNetFromCaffe("age_deploy.prototxt", "age_net.caffemodel")
gender_net = cv2.dnn.readNetFromCaffe("gender_deploy.prototxt", "gender_net.caffemodel")


In [11]:
# Define the age groups and gender labels

age_groups = ['1-5', '6-10', '11-15', '16-20', '21-25', '26-40', '41-60', '61-80', '80-100']
gender_labels = ['Male', 'Female']


In [13]:
#Age and Gender Prediction Function

def predict_age_gender(face_img):
    # Convert the image to blob format to input to the DNN model
    blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), (78.42, 87.76, 114.896), swapRB=False)
    
    # Predict gender
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    gender = gender_labels[gender_preds[0].argmax()]
    
    # Predict age
    age_net.setInput(blob)
    age_preds = age_net.forward()
    age = age_groups[age_preds[0].argmax()]
    
    return gender, age


In [15]:
# Open webcam for real-time detection

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()  # Capture frame from webcam
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    faces = face_model.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)  # Detect faces

    # For each face detected, predict age and gender
    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]  # Crop the face region
        gender, age = predict_age_gender(face_img)  # Get predictions

        # Draw rectangle around the face and display age and gender
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        label = f"{gender}, {age}"
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    # Display the frame with annotations
    cv2.imshow("Age and Gender Detection", frame)

    # Exit when 'a' is pressed
    if cv2.waitKey(1) & 0xFF == ord('a'):
        break

cap.release()  # Release the webcam
cv2.destroyAllWindows()  # Close all OpenCV windows
